So, this is a NASDAQ dataset from kaggle, found here : https://www.kaggle.com/datasets/jacksoncrow/stock-market-dataset/data <br> Importantly, it's actually divided into three parts- a folder containing ETF data, a folder containing stock data, and a standalone csv containing metadata about symbols. The Stock and ETF data is organized into many CSV files by ticker, so our first priority will be collecting all of those individual CSVs into one file/dataframe for use. Also, as we'll see, almost all of the columns in the metadata are probably unhelpful- "market category", for instance, is more than half empty, as is "financial status" and "CQS symbol". "Security name" and "nasdaq traded" are also not worth keeping, since *all* of the securities are nasdaq traded, and name is just the name. "Listing exchange" could factor in, so we will be retrieving that though. Technically we could determine if a symbol is an ETF when we join everything together later, but it's probably easier to preserve it here. Lot size is also getting dropped- technically, there are 5 stocks which do have small lot sizes, but later we'll be removing thinly traded stocks anyway, so it's not overly relevant.

In [20]:
# informational prints are commented out, but feel free to uncomment and run them to get a better idea of what the dataframe is like
import pandas as pd
import os
meta= "./symbols_valid_meta.csv"
symbol_meta= pd.read_csv(meta)
# print(symbol_meta.info())
# print(symbol_meta["Round Lot Size"].value_counts())
# small_lots=symbol_meta[symbol_meta["Round Lot Size"].isin([10.0, 1.0])==True]
# print("small", small_lots)
symbol_meta= symbol_meta.drop(["Nasdaq Traded","Security Name", "Round Lot Size", "CQS Symbol", "NextShares", "Financial Status", "Test Issue", "Market Category", "NASDAQ Symbol"],axis=1)
print(symbol_meta.info())



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8049 entries, 0 to 8048
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Symbol            8049 non-null   object
 1   Listing Exchange  8049 non-null   object
 2   ETF               8049 non-null   object
dtypes: object(3)
memory usage: 188.8+ KB
None


In [ ]:
stock_path= "./stocks"
etf_path= "./etfs"

